In [1]:
from prediction_utils.cohorts.ascvd.starr.cohort import ASCVDCohort

In [2]:
config_dict = {
    'dataset_project': 'som-rit-phi-starr-prod',
    'dataset': 'starr_omop_cdm5_deid_1pcent_latest',
#     'dataset': 'starr_omop_cdm5_deid_latest',
    'rs_dataset_project': 'som-nero-phi-nigam-starr',
    'rs_dataset': 'temp_dataset',
    'years_history_required': 1,
    'max_observation_period_end_date': '2020-12-31',
    'min_age_in_years': 18,
}

In [3]:
label_config = {
    'ascvd_1yr': {
        'max_index_date': "2018-12-31",
        'event_followup_days': 365
    },
    'ascvd_5yr': {
        'max_index_date': '2014-12-31',
        'event_followup_days': 5*365.25
    },
    'ascvd_10yr': {
        'max_index_date': '2009-12-31',
        'event_followup_days': 10*365.25
    }
}

In [4]:
cohort_label = 'ascvd_1yr'
config_dict = {**config_dict, **label_config[cohort_label]}

In [5]:
cohort=ASCVDCohort(**config_dict)

/share/pi/nigam/envs/anaconda/envs/prediction_utils/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [6]:
cohort.get_base_query()

'\n            WITH visits AS (\n                SELECT t1.person_id, visit_occurrence_id, visit_concept_id, \n                    birth_datetime, visit_start_date, observation_period_start_date, observation_period_end_date,\n                    CAST(DATE_DIFF(CAST(visit_start_date AS DATE), CAST(birth_datetime AS DATE), DAY) AS FLOAT64) / 365.25 as age_in_years,\n                    CAST(DATE_DIFF(CAST(visit_start_date AS DATE), CAST(observation_period_start_date AS DATE), DAY) AS FLOAT64) / 365.25 as years_since_start,\n                    CAST(DATE_DIFF(CAST(observation_period_end_date AS DATE), CAST(visit_start_date AS DATE), DAY) AS FLOAT64) / 365.25 as years_until_end\n                FROM som-rit-phi-starr-prod.starr_omop_cdm5_deid_1pcent_latest.visit_occurrence t1\n                INNER JOIN som-rit-phi-starr-prod.starr_omop_cdm5_deid_1pcent_latest.person as t2\n                    ON t1.person_id = t2.person_id\n                INNER JOIN som-rit-phi-starr-prod.starr_omop_cdm5

In [7]:
# base_query_df = cohort.db.read_sql_query(cohort.get_base_query())

In [8]:
cohort.db.execute_sql_to_destination_table(
    cohort.get_transform_query_sampled(),
    "{rs_dataset_project}.{rs_dataset}.temp_sampled".format_map(
        cohort.config_dict
    )
)